In [2]:
%load_ext autoreload
%autoreload 2

In [11]:
import sys
sys.path.append('../../')

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from structs.models import CIFAR100Model


import os 
path = '/Volumes/Ayush_Drive/mnist/'

if os.path.exists(path):
    prefix = path
else:
    prefix = ''

torch.manual_seed(42)

# Hyperparameters
batch_size = 64
learning_rate = 0.001
epochs = 50
# device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device = 'cpu'
print(f"Using device: {device}")

Using device: cpu


# Train SAEs

In [4]:
from tqdm import tqdm
from structs.models import EnhancedSAE, SimpleSAE
import torch.optim as optim

def train_sae(train_loader, input_dim, hidden_dim, device):
    model = SimpleSAE(input_dim=input_dim, hidden_dim=hidden_dim).to(device)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for activations in tqdm(train_loader, desc="Processing Activations"):
            activations = activations.to(device)
            optimizer.zero_grad()
            encoded, decoded = model(activations)
            loss = model.compute_loss(activations, decoded, encoded)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if epoch % 5 == 0:  # Save every 5 epochs
            checkpoint_path = os.path.join(prefix, 'embeddings', f'sae_depth_model_ckpt_{epoch}.pth')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }, checkpoint_path)


        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")
    
    return model

def test_sae(model, test_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for activations in tqdm(test_loader):
            activations = activations.to(device)
            encoded, decoded = model(activations)
            loss = model.compute_loss(activations, decoded, encoded)
            total_loss += loss.item()
    
    print(f"Test Loss: {total_loss/len(test_loader):.4f}")
    
    return total_loss/len(test_loader)

In [17]:
data_name = 'laion'
def infinite_saes(model, depth, custom_depth=10, dataset=None): 
    # the model provides the data 
    if depth > custom_depth: 
        print('Finished Training up to depth ', depth)
        return model

    # get the model weights 
    if dataset is None:
        print('Getting the model weights')
        dataset = model.decoder.weight.T.detach().clone()
    else:
        print('Getting the dataset weights')
        dataset = dataset

    #train and test set 

    train_weights, test_weights = train_test_split(dataset, test_size=0.2, random_state=42)
    train_loader = DataLoader(train_weights, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_weights, batch_size=batch_size, shuffle=False)
    
    # get the input and hidden dimensions
    input_dim = dataset.shape[1]
    hidden_dim = max(int(dataset.shape[0] / 2), 2304)  # Adjust as needed

    print('Creating SAE with input dim:', input_dim, 'and hidden dim:', hidden_dim)
    # create the model
    sae_model = train_sae(train_loader, input_dim, hidden_dim, device)
    loss = test_sae(sae_model, test_loader, device)

    del model 
    
    with open('log_weights.txt', 'a') as f:
        f.write(f"Data: {data_name} Depth: {depth}, Loss: {loss}\n")



    model_path = os.path.join(prefix, 'embeddings', data_name, f'sae_depth_{depth}_decoder.pth')
    torch.save(sae_model.state_dict(), model_path)
    print(f"SAE model saved to {model_path}")

    # Recursively call the function for the next depth
    return infinite_saes(sae_model, depth + 1, custom_depth=custom_depth)

- layer3 activations.shape = torch.Size([60000, 16384])
- layer4 activations.shape = 

In [15]:
sae_weights = torch.load(os.path.join(path, 'laion', 'sae_depth_1_weights.pt'), map_location=device)
sae_weights.shape

torch.Size([65536, 1024])

In [18]:
depth_1 = torch.load(os.path.join(path, 'embeddings', 'laion', 'sae_depth_2_decoder.pth'), map_location=device)
depth_1['decoder.weight'].shape

torch.Size([1024, 16384])

In [19]:
infinite_saes(None, 3, custom_depth=10, dataset=depth_1['decoder.weight'].T.detach().clone())

Getting the dataset weights
Creating SAE with input dim: 1024 and hidden dim: 8192


Processing Activations: 100%|██████████| 205/205 [00:06<00:00, 31.13it/s]


Epoch [1/50], Loss: 18.7885


Processing Activations: 100%|██████████| 205/205 [00:06<00:00, 32.02it/s]


Epoch [2/50], Loss: 0.0526


...

Processing Activations: 100%|██████████| 29/29 [00:00<00:00, 114.46it/s]


Epoch [50/50], Loss: 0.0001


100%|██████████| 8/8 [00:00<00:00, 581.47it/s]


Test Loss: 0.4180
SAE model saved to /Volumes/Ayush_Drive/mnist/embeddings/laion/sae_depth_10_decoder.pth
Finished Training up to depth  11


SimpleSAE(
  (encoder): Linear(in_features=1024, out_features=2304, bias=True)
  (decoder): Linear(in_features=2304, out_features=1024, bias=True)
)

In [ ]:
# # load the initial sae model 
# input_dim = 16384
# hidden_dim = 8192  # Adjust as needed
# sae_model = SimpleSAE(input_dim=input_dim, hidden_dim=hidden_dim)
# sae_model.load_state_dict(torch.load(f'{prefix}/embeddings/cifar100/sae_layer3_depth_1_{hidden_dim}.pth', map_location=torch.device('mps')))

In [ ]:
sae_weights = torch.load(os.path.join(path, 'laion', 'sae_depth_1_weights.pt'), map_location=device)
sae_weights.shape